In [1]:
#!pip install -U databricks-cli

In [2]:
#!conda install -y gwerbin databricks-cli

In [3]:
!%AddDeps com.databricks spark-csv_2.10 1.2.0 --transitive

/bin/sh: line 0: fg: no job control


In [4]:
import findspark
import datetime
findspark.init()
from pyspark import SparkContext
sc = SparkContext("local", "first app")
#print(sc)

In [5]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import DateType, IntegerType
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

sqlContext = SQLContext(sc)

In [6]:
winnersdata=sc.textFile("hdfs://quickstart.cloudera:8020/data/worldcup/teams")

In [7]:
winners=winnersdata.map(lambda line: (line.split(',')[0], 
                                      line.split(',')[1],line.split(',')[2],
                                      line.split(',')[3],line.split(',')[4],
                                      line.split(',')[5],line.split(',')[6],
                                      line.split(',')[7],line.split(',')[8],
                                      line.split(',')[9])).collect()


In [8]:
df_winners = sqlContext.createDataFrame(winners, schema=['Edition','Year','Host_Country','Winner','Runner_up', 
                                                         'Average_attendance' ,'Teams','Matches','Goals_scored',
                                                         'Average_goals'])
df_winners.show(3)

+--------------------+----+------------+-------+--------------+--------------------+-----+-------+------------+-------------+
|             Edition|Year|Host_Country| Winner|     Runner_up|  Average_attendance|Teams|Matches|Goals_scored|Average_goals|
+--------------------+----+------------+-------+--------------+--------------------+-----+-------+------------+-------------+
|             Edition|Year|Host_Country| Winner|     Runner_up| Average_attendance |Teams|Matches|Goals_scored|Average_goals|
|1930 World Cup Ur...|1930|     Uruguay|Uruguay|     Argentina|               32808|   13|     18|          70|          3.9|
|1934 World Cup Italy|1934|       Italy|  Italy|Czechoslovakia|               21353|   16|     17|          70|          4.1|
+--------------------+----+------------+-------+--------------+--------------------+-----+-------+------------+-------------+
only showing top 3 rows



In [9]:
wcheader = winnersdata.first()
winnersdataf=winnersdata.filter(lambda line: line != wcheader)
winnersrdd=winnersdataf.map(lambda line: (line.split(',')[0], line.split(',')[1],
                                          line.split(',')[2],line.split(',')[3],
                                          line.split(',')[4],line.split(',')[5],
                                          line.split(',')[6],line.split(',')[7],
                                          line.split(',')[8],line.split(',')[9])).collect()
df_winners = sqlContext.createDataFrame(winnersrdd, schema=['Edition','Year','Host_Country','Winner','Runner_up', 
                                                            'Average_attendance' ,'Teams','Matches','Goals_scored',
                                                            'Average_goals'])
df_winners.show(3)
df_winners.printSchema()

+--------------------+----+------------+-------+--------------+------------------+-----+-------+------------+-------------+
|             Edition|Year|Host_Country| Winner|     Runner_up|Average_attendance|Teams|Matches|Goals_scored|Average_goals|
+--------------------+----+------------+-------+--------------+------------------+-----+-------+------------+-------------+
|1930 World Cup Ur...|1930|     Uruguay|Uruguay|     Argentina|             32808|   13|     18|          70|          3.9|
|1934 World Cup Italy|1934|       Italy|  Italy|Czechoslovakia|             21353|   16|     17|          70|          4.1|
|1938 World Cup Fr...|1938|      France|  Italy|       Hungary|             20872|   15|     18|          84|          4.7|
+--------------------+----+------------+-------+--------------+------------------+-----+-------+------------+-------------+
only showing top 3 rows

root
 |-- Edition: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Host_Country: string (

In [10]:
df_winners.write.format("com.databricks.spark.avro").save("hdfs://quickstart.cloudera:8020/data/avro/winners")

In [12]:
coachesdata=sc.textFile("hdfs://quickstart.cloudera:8020/data/worldcup/coaches")

In [14]:
cheader = coachesdata.first()
coachesdataf=coachesdata.filter(lambda line: line != wcheader)
coachedrdd=coachesdataf.map(lambda line: (line.split('|')[0], line.split('|')[1])).collect()
df_coaches = sqlContext.createDataFrame(coachedrdd, schema=['Country','Coach'])
df_coaches.show(3)
df_coaches.printSchema()

+---------+---------------+
|  Country|          Coach|
+---------+---------------+
|  Country|          Coach|
|  Algeria| Rabah SAADANEH|
|Argentina|Diego MARADONAH|
+---------+---------------+
only showing top 3 rows

root
 |-- Country: string (nullable = true)
 |-- Coach: string (nullable = true)



In [15]:
df_coaches.write.parquet("hdfs://quickstart.cloudera:8020/data/parquet/coaches")

In [17]:
playersrdd = sc.wholeTextFiles("hdfs://quickstart.cloudera:8020//data/worldcup/players") \
               .map(lambda x: x[1])
df_players = sqlContext.read.json(playersrdd)
df_players.show(5)

+--------------------+-----------+-----------+-----------+-----------------+---------+------+--------+---------+----+
|                Club|ClubCountry|Competition|DateOfBirth|         FullName|IsCaptain|Number|Position|     Team|Year|
+--------------------+-----------+-----------+-----------+-----------------+---------+------+--------+---------+----+
|Club AtlÃ©tico Ta...|  Argentina|  World Cup|   1905-5-5|     Ãngel Bossio|    false|      |      GK|Argentina|1930|
|Quilmes AtlÃ©tico...|  Argentina|  World Cup| 1908-10-23|     Juan Botasso|    false|      |      GK|Argentina|1930|
|          Boca Junio|  Argentina|  World Cup|  1907-2-23|   Roberto Cherro|    false|      |      FW|Argentina|1930|
|Central Norte TucumÃ|  Argentina|  World Cup|  1907-2-23|Alberto Chividini|    false|      |      DF|Argentina|1930|
|Club Atletico Est...|  Argentina|  World Cup|  1909-3-19|                 |    false|    10|      FW|Argentina|1930|
+--------------------+-----------+-----------+----------

In [18]:
df_players.write.parquet("hdfs://quickstart.cloudera:8020/data/parquet/players")

In [19]:
df_avro=sqlContext.read.format("com.databricks.spark.avro") \
                  .load("hdfs://quickstart.cloudera:8020/data/avro/winners")
df_avro.show(3)

+--------------------+----+------------+-------+--------------+------------------+-----+-------+------------+-------------+
|             Edition|Year|Host_Country| Winner|     Runner_up|Average_attendance|Teams|Matches|Goals_scored|Average_goals|
+--------------------+----+------------+-------+--------------+------------------+-----+-------+------------+-------------+
|1930 World Cup Ur...|1930|     Uruguay|Uruguay|     Argentina|             32808|   13|     18|          70|          3.9|
|1934 World Cup Italy|1934|       Italy|  Italy|Czechoslovakia|             21353|   16|     17|          70|          4.1|
|1938 World Cup Fr...|1938|      France|  Italy|       Hungary|             20872|   15|     18|          84|          4.7|
+--------------------+----+------------+-------+--------------+------------------+-----+-------+------------+-------------+
only showing top 3 rows



In [20]:
df_parquet=sqlContext.read.parquet("hdfs://quickstart.cloudera:8020/data/parquet/coaches")
df_parquet.show(3)


+---------+---------------+
|  Country|          Coach|
+---------+---------------+
|  Country|          Coach|
|  Algeria| Rabah SAADANEH|
|Argentina|Diego MARADONAH|
+---------+---------------+
only showing top 3 rows

